In [60]:
%pip install groq
%pip install python-pptx

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [61]:
query="effect of demonitatization in india"

In [62]:
from groq import Groq

client = Groq(api_key="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")
completion = client.chat.completions.create(
    model="llama3-8b-8192",
    messages=[
        {
            "role": "system",
            "content": "generate / rectify grammar - give a one line response - neither too long nor too short (maximum 5 words) - just the title"
        },
        {
            "role": "user",
            "content": f"query[{query}]"
        }
    ],
    temperature=1,
    max_tokens=8192,
    top_p=1,
    stream=True,
    stop=None,
)
query=""
for chunk in completion:
    query+=chunk.choices[0].delta.content or ""
print(query)

Impact of Demonetization in India


In [63]:
slides=5

In [64]:
completion = client.chat.completions.create(
    model="llama3-8b-8192",
    messages=[
        {
            "role": "system",
            "content": f"generate {slides} ppt slide titles in order (keep them sweet and short) based on the query[] - split the titles with delimiter '%%%' and return all titles within a codeblock ``` - do not mention slide count - do not include asterisks * - dont generate question tags - max words - (6 to 8) - dont include double quotes ,just plain text"
        },
        {
            "role": "user",
            "content": f"query[{query}]"
        }
    ],
    temperature=1,
    max_tokens=8192,
    top_p=1,
    stream=True,
    stop=None,
)
slide_points_full=""
for chunk in completion:
    slide_points_full+=chunk.choices[0].delta.content or ""


In [65]:
print(slide_points_full)

```
Impact of Demonetization in India %%%
RBI's Role in Demonetization %%%
Economic Impact of Note Ban %%%
Public Perception of Demonetization %%%
Long Term Effects of Demonetization
```


In [66]:
slide_points=slide_points_full[slide_points_full.find('```')+4:slide_points_full.rfind('```')-1].replace('\n',"").split("%%%")

In [67]:
print(slide_points)

['Impact of Demonetization in India ', "RBI's Role in Demonetization ", 'Economic Impact of Note Ban ', 'Public Perception of Demonetization ', 'Long Term Effects of Demonetization']


In [68]:
slide_content=[]

In [69]:
for point in slide_points:
    if point.strip()!="":
        completion = client.chat.completions.create(
            model="llama3-8b-8192",
            messages=[
                {
                    "role": "system",
                    "content": f"generate a paragraph (50-60 words STRICTLY) for slide content ONLY (neither too big nor too small) for title[] - return the body only without any unwanted text **do not use linebreak** - do not use asterisks* - use justify text align - do not mention anything else, give only the slide content paragraph as a response - return only information related to the point[] not statements like 'here is..','some of ...' etc - do not use linebreak generate entire response in one line - dont use asterisks * "
                },
                {
                    "role": "user",
                    "content": f"[{point}]"
                }
            ],
            temperature=1,
            max_tokens=8192,
            top_p=1,
            stream=True,
            stop=None,
        )
        temp_slide_content=""
        for chunk in completion:
            temp_slide_content+=chunk.choices[0].delta.content or ""
        slide_content.append(temp_slide_content)

In [70]:
print(slide_content)

['The demonetization of high-denomination currency notes in November 2016 led to a severe cash crunch resulting in long queues and economic disruption. Many businesses were forced to shut down, while others witnessed a significant decline in sales. The move aimed to curb black money and terrorism financing, but its effectiveness in achieving these goals remains uncertain.', "The Reserve Bank of India played a pivotal role in executing the demonetization drive, withdrawing legal tender status from high-denomination banknotes of Rs 500 and Rs 1000, seizing 86% of the country's currency stock, and distributing the new Rs 2000 note.", 'The economic impact of demonetization was felt across various sectors as the sudden withdrawal of high denomination notes resulted in a severe cash crunch affecting small businesses and daily transactions. GDP growth slowed down, and the economic output contracted in the quarter following the note ban. Additionally, inflation increased as supply chain disrup

In [71]:
from pptx import Presentation
from pptx.util import Inches
from PIL import Image
from pptx.dml.color import RGBColor

slide_width = Inches(10)  
slide_height = Inches(5.625) 

prs = Presentation()

prs.slide_width = slide_width
prs.slide_height = slide_height

slide = prs.slides.add_slide(prs.slide_layouts[5])  

img_path = 'styles/template1/1.png'
img = Image.open(img_path)
img_width, img_height = img.size

max_width = slide_width  
max_height = slide_height 
scale_width = max_width / img_width
scale_height = max_height / img_height
scale = min(scale_width, scale_height)

left = (slide_width - img_width * scale) / 2
top = (slide_height - img_height * scale) / 2

pic = slide.shapes.add_picture(img_path, left, top, width=img_width * scale, height=img_height * scale)
title_width = Inches(8) 
title_left = (slide_width - title_width) / 2
title = slide.shapes.add_textbox(title_left, Inches(2), title_width, Inches(1))
title.text = query
title.text_frame.word_wrap = True
title.text_frame.paragraphs[0].font.color.rgb = RGBColor(255, 255, 255) 
title.text_frame.paragraphs[0].font.bold = True  
title.text_frame.paragraphs[0].font.size = Inches(0.4)  
title.text_frame.paragraphs[0].font.name = 'Courier New' 
title.text_frame.paragraphs[0].alignment = 2

In [72]:
img_path2 = 'styles/template1/2.png'
for count in range(len(slide_points)):
    slide = prs.slides.add_slide(prs.slide_layouts[5])  

    left = (slide_width - img_width * scale) / 2
    top = (slide_height - img_height * scale) / 2

    pic = slide.shapes.add_picture(img_path2, left, top, width=img_width * scale, height=img_height * scale)

    title = slide.shapes.add_textbox(left, Inches(0.2), img_width * scale, Inches(1))
    title.text = slide_points[count]
    title.text_frame.word_wrap = True
    title.text_frame.paragraphs[0].font.color.rgb = RGBColor(255, 255, 255) 
    title.text_frame.paragraphs[0].font.bold = True
    title.text_frame.paragraphs[0].font.size = Inches(0.3) 
    title.text_frame.paragraphs[0].font.name = 'Courier New' 
    title.text_frame.paragraphs[0].alignment = 2
    title = slide.shapes.add_textbox(left+Inches(0.25), Inches(1.5), Inches(4), Inches(1))
    title.text = slide_content[count]
    title.text_frame.word_wrap = True  
    title.text_frame.paragraphs[0].font.color.rgb = RGBColor(255, 255, 255) 
    title.text_frame.paragraphs[0].font.bold = False
    title.text_frame.paragraphs[0].font.size = Inches(0.15) 
    title.text_frame.paragraphs[0].font.name = 'Courier New' 
    title.text_frame.paragraphs[0].alignment = 2

prs.save('test.pptx')

In [73]:
%pip install -qU langchain-text-splitters
%pip install pypdf
%pip install sentence_transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [74]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
    separators=["\n\n", "\n", " ", ""],
)
from pypdf import PdfReader

reader = PdfReader("test.pdf")
pages=len(reader.pages)
document_data=""
for i in range(pages):
    document_data+=" "+reader.pages[i].extract_text()
texts = text_splitter.create_documents([document_data])
texts2=[]
for i in texts:
    texts2.append(i.page_content)


In [75]:
print(texts2)

['Essential  Drugs  List - Ayurveda  \n \n(A) Asava Arista  \n \nSr \nNo Name of \nMedicine  Refe\nrenc\ne', 'renc\ne \ntext Dispensi\nng Pack \nSize Main Indications  Dose  Precaution/ \nContrain dicati', 'Contrain dicati\non Preferred \nuse (OPD/ \nIPD)  \n1.  Abhayarishta  AFI 200 ml  Arsha, Agnimandya,', 'Udararoga, Vibandha  12   -   24   ml  NS Both', '2.  Amritarishta  AFI 200 ml  Sarva Jvara , Jirna Jvara  12   -   24   ml  NS Both', '3.  Aragvadharishta  AH 200ml  Kandu, Tvak  Vikar a, \nVibandha  12   -   24   ml  NS Both', '4.  Aravindasava  AFI 200 ml  Balaroga, Balakshaya,', 'Agnimandya, Aruchi  12   -   24   ml  NS Both  \n5.  Arjunarishta/', '5.  Arjunarishta/ \nParthadyarishta  AFI 200 ml  Hridroga, Hriddrav a, Hrid -\ndaurbalya, Moha,', 'daurbalya, Moha, \nMurc hha 12   -   24   ml  NS Both', '6.  Ashokarishta  AFI 200 ml  Asrigdara, Shveta \nPradara,  Yoniroga  12   -   24   ml  NS Both', '7.  Ashvagandharisht\na AFI 200 ml  Murc hha, Apasmara,', 'Shosha, Unmada, Kars

In [76]:
%pip install langchain_community

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [77]:
from langchain_community.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
doc_result = embeddings.embed_documents(texts2)

In [78]:
vectors = []

for i in range(len(doc_result)):
    vectors.append(
        {
            "id": f"vec{i+1}", 
            "values": doc_result[i], 
            "metadata": {'text': texts2[i]}
        }
    )


In [79]:
%pip install pinecone_client

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [80]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="d1eccc8c-3147-4d5e-992e-81b0c5d73278")
pc.create_index(
    name="quickstart",
    dimension=384, 
    metric="euclidean",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [81]:
index = pc.Index("quickstart")
index.upsert(
    vectors=vectors,
    namespace= "ns1"

)

{'upserted_count': 371}

In [82]:
from sentence_transformers import SentenceTransformer
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2', device=device)
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [83]:
query = """what medicine should i eat or drink for Balaroga, Balakshaya, 
Agnimandya, Aruchi """

In [84]:
xq = model.encode(query).tolist()
xc=index.query(
    namespace="ns1",
    vector=xq,
    top_k=10,
    include_values=True,
    include_metadata=True
)
relevant_chunk=""
for result in xc['matches']:
    relevant_chunk+=" "+result['metadata']['text']
print(relevant_chunk)

 Dhatukshaya, 
Sutikaroga, Balaroga  10-30 drops  for 
internal use  Amadosh
a Both 4.  Aravindasava  AFI 200 ml  Balaroga, Balakshaya, Jirnajvara, Daha  500 mg - 1 gm  with 
ghee, sugar, warm 
milk or water  NS Both Essential  Drugs  List - Ayurveda  
 
(A) Asava Arista  
 
Sr 
No Name of 
Medicine  Refe
renc
e Palitya  2-10 drops  for  nasal 
administration  NS Both  
117.  Apamargkshara Urustambha, 
Pakshaghata,G ridhra si 250 mg  Long term 
use, Vrikka 
Roga, peri - 
concepti 
onal (Mukhroga)  AFI 10 gm  Mukhdaurgandhya, 
Mukh apaka, Dantaroga, 
Galaroga  1 - 2 pills  NS Both Mutraghata , Gulma  250-375 mg  
 NS Both  
(M) Varti, Netrabindu, Anjana  
 
Sr Vikara or 
Prakriti, 
Raktaj 
Roga, 
pregnancy
, long  
term use  Both NS Both  
(S) Rasayoga  
 
Sr 
No Name of Medicine  Refere
nce 
Text  Dispen
sing 
Pack


In [85]:
completion = client.chat.completions.create(
    model="llama3-8b-8192",
    messages=[
        {
            "role": "system",
            "content": f"answer the query[] based on the relevant_chunk[]"
        },
        {
            "role": "user",
            "content": f"query[{query}] relevant_chunk[{relevant_chunk}]"
        }
    ],
    temperature=0.1,
    max_tokens=8192,
    top_p=1,
    stream=True,
    stop=None,
)
relevant_response=""
for chunk in completion:
    relevant_response+=chunk.choices[0].delta.content or ""
print(relevant_response)

Based on the relevant chunk, here are the answers to the query:

For Balaroga (Weakness):

* 10-30 drops of the medicine for internal use (Amadosh)

For Balakshaya (Wasting):

* 10-30 drops of the medicine for internal use (Amadosh)

For Agnimandya (Loss of Appetite):

* 500 mg - 1 gm of the medicine with ghee, sugar, warm milk or water

For Aruchi (Lack of Taste):

* 500 mg - 1 gm of the medicine with ghee, sugar, warm milk or water


In [86]:
pc.delete_index('quickstart')